In [3]:
import json
import numpy as np
from utils import tokenize,stem,bow
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from model import NeuralNet

In [4]:
# Reading Intentions
with open('intentions.json','r') as file:
    intentions = json.load(file)

all_words,tags,xy = [],[],[]

# Ignore Words
ignore = ['?',',','!','.','-','\'']

In [5]:
# Tokenizing & Stemming to get all words and tags 
for intent in intentions['intentions']:
    tag = intent['tag']
    tags.append(tag)

    for pattern in intent['patterns']:
        tokenied_sentence = tokenize(pattern)
        xy.append((tokenied_sentence, tag))

        stemmed_words = [stem(w) for w in tokenied_sentence if w not in ignore]
        all_words.extend(stemmed_words)

all_words = sorted(set(all_words))
tags = sorted(set(tags))

In [6]:
# Using Bag of Words (BOW) to create X_train and y_train
X_train, y_train = [],[]
for (tokenied_sentence, tag) in xy:
    bag = bow(tokenied_sentence, all_words)
    X_train.append(bag)
    y_train.append(tags.index(tag))
    
X_train, y_train = np.array(X_train), y_train

In [7]:
# Pytorch Dataset preparation. This will be used in Pytorch model.
class MyDataset(Dataset):
    def __init__(self):
        self.n_samples = len(X_train)
        self.x_data = X_train
        self.y_data = y_train

    def __getitem__(self, i):
        return self.x_data[i], self.y_data[i]    

    def __len__(self):
        return self.n_samples

In [8]:
# Defining model parameters.
input_size = len(all_words)
hidden_size = 16
ouput_size = len(tags)
lr = 0.001
num_epochs = 1000
device = torch.device ('cuda' if torch.cuda.is_available() else 'cpu')

In [9]:
# Setting dataset to DataLoader (Used by Pytroch), Defining model Loss, Defining model optimizer
dataset = MyDataset()
train_loader = DataLoader(dataset=dataset, batch_size=8, shuffle=True, num_workers=2)

model = NeuralNet(input_size, hidden_size, ouput_size)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [10]:
# Training the model
for epoch in range(num_epochs):
    for (X, y) in train_loader:
        X = X.to(device) # GPU if GPU exists else CPU
        y = y.to(device) # GPU if GPU exists else CPU

        y_hat = model(X) # predictions
        loss = criterion(y_hat, y) # loss calculation from predictions and true labels

        optimizer.zero_grad() # Making gradients zero for next iteration
        loss.backward() # Changing weights and bias based in loss and optimizer.step()
        optimizer.step()

    if (epoch+1)%100 == 0:
        print("Epoch : ",epoch+1, "/", num_epochs," loss : ", loss.item())

print("Final Loss : ", loss.item())

Epoch :  100 / 1000  loss :  1.0670640468597412
Epoch :  200 / 1000  loss :  0.7534254789352417
Epoch :  300 / 1000  loss :  0.6860737800598145
Epoch :  400 / 1000  loss :  0.6405797004699707
Epoch :  500 / 1000  loss :  0.26428890228271484
Epoch :  600 / 1000  loss :  0.04215848445892334
Epoch :  700 / 1000  loss :  0.11291855573654175
Epoch :  800 / 1000  loss :  0.056964874267578125
Epoch :  900 / 1000  loss :  0.18366849422454834
Epoch :  1000 / 1000  loss :  0.07135498523712158
Final Loss :  0.07135498523712158


In [11]:
# Saving the model for chatbot
data  = {
    "model_state":model.state_dict(),
    "input_size":input_size,
    "output_size":ouput_size,
    "hidden_size":hidden_size,
    "all_words":all_words,
    "tags":tags
}

torch.save(data, "saved_model.pth")